# 日本国債損益モニタリング

## ライブラリインポート

In [1]:
import numpy as np
import pandas as pd
import QuantLib as ql
import re

ModuleNotFoundError: No module named 'numpy'

## 明細読み込み

In [ ]:
historical_meisai = pd.read_csv('Input//日本国債明細.csv')
historical_meisai['基準日'] = pd.to_datetime(historical_meisai['基準日'], format='%Y/%m/%d')
historical_meisai['発行日'] = pd.to_datetime(historical_meisai['発行日'], format='%Y/%m/%d')
historical_meisai['償還日'] = pd.to_datetime(historical_meisai['償還日'], format='%Y/%m/%d')
historical_meisai.set_index(['基準日', '銘柄'], inplace=True)

eval_dates = historical_meisai.index.get_level_values('基準日').unique()

display(historical_meisai)

発行日        償還日       利率           簿価
基準日        銘柄                                                          
2024-04-30 第437回利付国庫債券（2年）  2022-06-01 2024-06-01 -0.00058    400000000
           第443回利付国庫債券（2年）  2022-12-01 2024-12-01  0.00029    200000000
           第448回利付国庫債券（2年）  2023-05-01 2025-05-01  0.00042    300000000
           第142回利付国庫債券（5年）  2020-03-11 2024-12-20 -0.00220   6000000000
           第153回利付国庫債券（5年）  2022-09-14 2027-06-20  0.00040  10000000000
...                                ...        ...      ...          ...
2025-03-31 第335回利付国庫債券（10年） 2014-09-22 2024-09-20  0.00517            0
           第347回利付国庫債券（10年） 2017-08-03 2027-06-20  0.00073   5000000000
           第375回利付国庫債券（10年） 2024-09-04 2034-06-20  0.00915   1000000000
           第95回利付国庫債券（20年）  2007-07-31 2027-06-20  0.02293            0
           第145回利付国庫債券（20年） 2013-06-20 2033-06-20  0.01686            0

[137 rows x 4 columns]

## 日本国債コンベンション

In [ ]:
tenor = ql.Period(ql.Semiannual)  # 利払間隔
calendar = ql.Japan()             # 休祝日カレンダー
convention = ql.ModifiedFollowing # 営業日調整
day_count = ql.Actual360()        # 日数計算
rule = ql.DateGeneration.Backward # 日付生成のルール
end_of_month = False              # 月末日ロール

settlemant_days = 1  # 決済日数

## 和暦を西暦に変換する関数の定義

In [ ]:
def convert_japanese_era_to_gregorian(date_str: str) -> str | None:
    """和暦の日付を西暦に変換する関数"""
    
    # 元号と対応する西暦の開始年
    era_map: dict[str, int] = {
        "S": 1925,  # 昭和1年は1926年
        "H": 1988,  # 平成1年は1989年
        "R": 2018   # 令和1年は2019年
    }

    # 正規表現で元号・年・月・日を抽出
    match: re.Match | None = re.match(r"(S|H|R)(\d+)\.(\d+)\.(\d+)", date_str)
    
    if match:
        era: str
        year: int
        month: int
        day: int
        
        era, year_str, month_str, day_str = match.groups()
        year = int(year_str)
        month = int(month_str)
        day = int(day_str)

        # 西暦の計算
        gregorian_year: int = era_map[era] + year
        
        return f"{gregorian_year}.{month}.{day}"
    
    return None  # 変換できない場合

## JGB YTM読み込み

> 公表している金利はどのようなものですか
> 
> 【答】
> 流通市場における固定利付国債の実勢価格に基づいて算出した主要年限毎の半年複利金利（半年複利ベースの最終利回り）です。（財務省）

In [ ]:
historical_ytms = pd.read_csv('Input//jgbcm_all.csv', encoding='shift-jis', header=1)
historical_ytms['基準日'] = historical_ytms['基準日'].apply(convert_japanese_era_to_gregorian)
historical_ytms['基準日'] = pd.to_datetime(historical_ytms['基準日'], format='%Y.%m.%d')
historical_ytms.set_index('基準日', inplace=True)

display(historical_ytms)

,1年,2年,3年,4年,5年,6年,7年,8年,9年,10年,15年,20年,25年,30年,40年
基準日,,,,,,,,,,,,,,,
1974-09-24,10.327,9.362,8.83,8.515,8.348,8.290,8.240,8.121,8.127,-,-,-,-,-,-
1974-09-25,10.333,9.364,8.831,8.516,8.348,8.290,8.240,8.121,8.127,-,-,-,-,-,-
1974-09-26,10.34,9.366,8.832,8.516,8.348,8.290,8.240,8.122,8.128,-,-,-,-,-,-
1974-09-27,10.347,9.367,8.833,8.517,8.349,8.290,8.240,8.122,8.128,-,-,-,-,-,-
1974-09-28,10.354,9.369,8.834,8.518,8.349,8.291,8.240,8.122,8.129,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-26,0.554,0.728,0.808,0.927,1.023,1.075,1.151,1.264,1.395,1.522,2.136,2.498,2.758,2.9,3.24
2025-05-27,0.566,0.739,0.811,0.922,1.012,1.057,1.117,1.229,1.357,1.479,2.035,2.354,2.613,2.743,3.062
2025-05-28,0.575,0.753,0.83,0.955,1.054,1.108,1.179,1.286,1.409,1.531,2.077,2.406,2.656,2.796,3.107


JGBイールドカーブを引く

In [ ]:
list_meisai = []

for eval_date in eval_dates:

    rates = historical_ytms.loc[eval_date]
    dict_rates = rates.to_dict()
    dict_rates = {int(k[:-1]): np.float64(v) for k, v in rates.items()} # 年限を整数に変換し、値をfloat64に変換

    eval_date_ql = ql.Date.from_date(eval_date)
    ql.Settings.instance().evaluationDate = eval_date_ql

    helpers = []
    for tenor, rate in dict_rates.items():
        # quote = ql.SimpleQuote(rate / 100)
        # helper = ql.DepositRateHelper(
        #     ql.QuoteHandle(quote),
        #     ql.Period(tenor, ql.Years),
        #     settlemant_days,
        #     calendar,
        #     convention,
        #     day_count,
        #     ql.Compounded,  # 利率の複利計算方法
        #     ql.Semiannual,  # 利率の複利計算の頻度
        #     False           # 単利フラグ（False: 複利）
        # )
        
        # 評価日スタートの仮想的な利払いスケジュールを作成
        schedule = ql.Schedule(
            eval_date_ql,
            calendar.adjust(eval_date_ql + ql.Period(tenor, ql.Years), ql.ModifiedFollowing),
            ql.Period(ql.Semiannual), 
            ql.Japan(), 
            ql.ModifiedFollowing,
            ql.ModifiedFollowing,
            ql.DateGeneration.Backward, 
            False
        )
    

        # # 仮想的な債券オブジェクトを定義
        # bond = ql.FixedRateBond(settlemant_days,  # 決済日数
        #                         100.0,  # 額面
        #                         schedule,  # 債券のスケジュール
        #                         [rate / 100],  # クーポン率
        #                         day_count)  # 日数計算方法
        # interest_rate = ql.InterestRate(rate / 100, day_count, ql.Compounded, ql.Semiannual)
        # price = bond.cleanPrice(interest_rate, day_count, ql.Compounded, ql.Semiannual)

        price = 100.
        price_quote = ql.SimpleQuote(price)
        price_handle = ql.QuoteHandle(price_quote)
        
        helper = ql.FixedRateBondHelper(
            price_handle,  # 債券価格
            1,  # 決済日数
            100.0,  # 額面
            schedule,  # 債券のスケジュール
            [rate / 100],  # クーポン率
            day_count,  # 日数計算方法
        )

        helpers.append(helper)
    
    discount_curve = ql.PiecewiseLogLinearDiscount(eval_date_ql, helpers, day_count)
    # discount_curve.enableExtrapolation()
    discount_handle = ql.YieldTermStructureHandle(discount_curve)
    engine = ql.DiscountingBondEngine(discount_handle)

    meisai = historical_meisai.loc[eval_date].copy()
    for _ in range(len(meisai)):
        
        meigara = meisai.iloc[_]
        schedule = ql.Schedule(
            ql.Date.from_date(meigara['発行日']),
            ql.Date.from_date(meigara['償還日']),
            ql.Period(ql.Semiannual),
            calendar,
            convention,
            convention,
            rule,
            end_of_month
            )

        bond = ql.FixedRateBond(
                settlemant_days,
                float(meigara['簿価']),
                schedule,
                [meigara['利率']],
                day_count
            )
        bond.setPricingEngine(engine)
            
        meisai.loc[meisai.index[_], '時価'] = bond.NPV()

    meisai['基準日'] = eval_date
    meisai.set_index('基準日', append=True, inplace=True)
    meisai = meisai.swaplevel(0, 1)
    list_meisai.append(meisai)

In [ ]:
pd.concat(list_meisai).to_csv('Output//国内債券明細_時価評価.csv', encoding='utf-8')